# Backpropagation

(예제) XOR 문제 딥러닝 모델 학습

In [25]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [49]:
# 1. XOR 데이터
X = torch.tensor([[0,0],[0,1],[1,0],[1,1]], dtype=torch.float)
Y = torch.tensor([[0],[1],[1],[0]], dtype=torch.float)
X

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])

In [ ]:
# 2. 모델 정의
torch.manual_seed(530)

model = nn.Sequential(
    nn.Linear(2, 2),    # 은닉층 : 입력 2 -> 출력 2 랜덤 초기화
    nn.ReLU(),          # Relu 활성화 함수
    nn.Linear(2, 1),    # 출력층(마지막) : 입력 2 -> 출력 1 랜덤 초기화
    nn.Sigmoid()        # Sigmoid 활성화 함수 (확률처럼)
)

loss_fn = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.1)
model

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
  (3): Sigmoid()
)

In [46]:
# 3. 학습
records = []

for epoch in range(100000):
    output = model(X)           # 예측값 계산
    loss = loss_fn(output, Y)   # 손실 계산

    optimizer.zero_grad()       # 기울기 초기화
    loss.backward()             # 역전파
    optimizer.step()            # 파라미터 업데이트

    if epoch % 10000 == 0 or epoch == 1:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
        records.append({
            "epoch": epoch,
            "hidden_weight": model[0].weight.detach().numpy().copy(),
            "hidden_bias": model[0].bias.detach().numpy().copy(),
            "output_weight": model[2].weight.detach().numpy().copy(),
            "output_bias": model[2].bias.detach().numpy().copy(),
        })

df_records = pd.DataFrame([
    {
        "Epoch": r["epoch"],
        "Hidden_W": r["hidden_weight"].round(3),
        "Hidden_b": r["hidden_bias"].round(3),
        "Output_W": r["output_weight"].round(3),
        "Output_b": r["output_bias"].round(3),
    }
    for r in records
])

df_records

Epoch 0, Loss: 0.7088
Epoch 1, Loss: 0.7058
Epoch 10000, Loss: 0.0017
Epoch 20000, Loss: 0.0008
Epoch 30000, Loss: 0.0005
Epoch 40000, Loss: 0.0004
Epoch 50000, Loss: 0.0003
Epoch 60000, Loss: 0.0002
Epoch 70000, Loss: 0.0002
Epoch 80000, Loss: 0.0002
Epoch 90000, Loss: 0.0002


,Epoch,Hidden_W,Hidden_b,Output_W,Output_b
0,0,"[[-0.399, -0.457], [0.606, 0.699]]","[0.655, -0.051]","[[-0.647, 0.661]]",[-0.176]
1,1,"[[-0.407, -0.465], [0.603, 0.696]]","[0.645, -0.046]","[[-0.647, 0.655]]",[-0.178]
2,10000,"[[-2.779, -2.779], [2.779, 2.779]]","[2.779, -2.779]","[[-4.763, -4.764]]",[5.855]
3,20000,"[[-2.942, -2.942], [2.941, 2.941]]","[2.942, -2.941]","[[-5.047, -5.048]]",[6.613]
4,30000,"[[-3.03, -3.03], [3.03, 3.03]]","[3.03, -3.03]","[[-5.202, -5.203]]",[7.046]
5,40000,"[[-3.091, -3.091], [3.09, 3.09]]","[3.091, -3.09]","[[-5.307, -5.308]]",[7.349]
6,50000,"[[-3.136, -3.136], [3.136, 3.136]]","[3.136, -3.136]","[[-5.387, -5.388]]",[7.583]
7,60000,"[[-3.173, -3.173], [3.172, 3.172]]","[3.173, -3.173]","[[-5.451, -5.452]]",[7.774]
8,70000,"[[-3.204, -3.204], [3.203, 3.203]]","[3.203, -3.203]","[[-5.504, -5.505]]",[7.934]
9,80000,"[[-3.23, -3.23], [3.229, 3.229]]","[3.23, -3.229]","[[-5.55, -5.551]]",[8.073]


In [ ]:
# 4. 결과 확인
with torch.no_grad():
    output = model(X)
    predicted = torch.round(output)  # 0.5 기준 반올림 (이진 분류)
    print("\n예측 결과:")
    print(predicted)


예측 결과:
tensor([[0.],
        [1.],
        [1.],
        [0.]])


### XOR 모델 학습 과정 정리 : Epoch 0 → 1

> ### 초기 파라미터 (Epoch 0)
- layer1 : 은닉층
- layer2 (마지막) : 출력층

$$
\mathbf{W}_1 = \begin{bmatrix}
-0.399 & -0.457 \\
0.606 & 0.699
\end{bmatrix}
$$

$$
\mathbf{b}_1 = \begin{bmatrix}
0.655 \\
-0.051
\end{bmatrix}
$$

$$
\mathbf{W}_2 = \begin{bmatrix}
-0.647 & 0.661
\end{bmatrix}
$$

$$
\mathbf{b}_2 = \begin{bmatrix}
-0.176
\end{bmatrix}
$$

---

> ### 순전파 (Forward)

**1. 입력 $x$ (4개 샘플, 4 by 2)**

$$
x =
\begin{bmatrix}
0 & 0 \\
0 & 1 \\
1 & 0 \\
1 & 1
\end{bmatrix}
$$

**2. 은닉층 선형합: $z_1 = x W_1^{T} + b_1$** <br>
(참고, xW와 b가 차원이 안 맞지만 토치의 브로드캐스팅 통해 연산 가능)

$$
z_1 = 
\begin{bmatrix}
0 & 0 \\
0 & 1 \\
1 & 0 \\
1 & 1
\end{bmatrix}
\begin{bmatrix}
-0.399 & 0.606 \\
-0.457 & 0.699
\end{bmatrix}
+
\begin{bmatrix}
0.655 & -0.051
\end{bmatrix}
$$

※ $b_1$는 shape이 $(2,)$이지만 **브로드캐스팅**에 의해 각 샘플마다 더해짐

$$
z_1 =
\begin{bmatrix}
0.655 & -0.051 \\
0.198 & 0.648 \\
0.256 & 0.555 \\
-0.201 & 1.254
\end{bmatrix}
$$

**3. ReLU 적용: $h = ReLU(z_1)$**

$$
h =
\begin{bmatrix}
0.655 & 0.000 \\
0.198 & 0.648 \\
0.256 & 0.555 \\
0.000 & 1.254
\end{bmatrix}
$$


**4. 출력층 선형합: $z_2 = h W_2^{T} + b_2$**

$$
z_2 = 
\begin{bmatrix}
0.655 & 0.000 \\
0.198 & 0.648 \\
0.256 & 0.555 \\
0.000 & 1.254
\end{bmatrix}
\begin{bmatrix}
-0.647 \\
0.661
\end{bmatrix}
+
\begin{bmatrix}
-0.176
\end{bmatrix}
=
\begin{bmatrix}
-0.5998 \\
0.1242 \\
0.0252 \\
0.6529
\end{bmatrix}
$$

**5. 출력값: $y_{pred} = sigmoid(z_2)$**

$$
\hat{y} =
\begin{bmatrix}
0.3544 \\
0.5310 \\
0.5063 \\
0.6577
\end{bmatrix}
$$


**6. 정답 $y$**

$$
y =
\begin{bmatrix}
0 \\
1 \\
1 \\
0
\end{bmatrix}
$$

---

> ### 손실 계산 (Loss)

- 여기선 **Binary Cross Entropy Loss** 사용 (정답과 예측이 다르면 손실이 커짐을 정답이 1,0일 때 조건부 수식으로 설명)  
      - 정답 y = 1일 떄, L = -\log(\hat{y}) : 정답이 1인데, 예측한 확률이 낮으면 손실이 크게 계산됨  
      - 정답 y = 0일 때, L = -\log(1 - \hat{y}) : 정답이 0인데, 예측한 확률이 높으면 손실이 크게 계산됨

\begin{aligned}
L &= -\frac{1}{N} \sum_{i=1}^N \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right] \\
&= -\frac{1}{4} \left[
(0)\log(0.3544) + (1)\log(1 - 0.3544) \right. \\
&\quad + (1)\log(0.5310) + (0)\log(1 - 0.5310) \\
&\quad + (1)\log(0.5063) + (0)\log(1 - 0.5063) \\
&\quad + (0)\log(0.6577) + (1)\log(1 - 0.6577) \left. \right] \\
&= -\frac{1}{4} \left[
\log(1 - 0.3544) + \log(0.5310) + \log(0.5063) + \log(1 - 0.6577)
\right] \\
&= -\frac{1}{4} \left[
\log(0.6456) + \log(0.5310) + \log(0.5063) + \log(0.3423)
\right] \\
&= 0.7058
\end{aligned}

---

> ### 역전파 (Backpropagation)
- y와 y_hat이 같기를 궁극적으로 목표
- gradient(미분, 기울기)를 활용하여 w와 b 업데이트  
      - ∂L / ∂W : w를 살짝 늘렸을 때, Loss가 얼마나 늘어나는지  
      - 기울기가 양수면, w를 줄여야 Loss도 감소 / 음수면, w를 늘려야 Loss가 감소 (-> 기울기*a 빼야함)
- 여러 레이어가 있을 때 기울기를 구하는 방식은 **Chain Rule**  
      - 출력 -> 입력 방향으로 미분을 계속 곱해나가며 역전파 수행  
      - e.g. $\frac{dL}{dx} = \frac{dL}{dy} \cdot \frac{dy}{dz} \cdot \frac{dz}{dx}$


**우리가 구해야 하는 파라미터는 w1, w2, b1, b2로, 그에 대한 gradient $\frac{\partial L}{\partial W_1}, \frac{\partial L}{\partial W_2}, \frac{\partial L}{\partial b_2}, \frac{\partial L}{\partial b_2}$를 구해야 함**

**0. 구조 다시 보기**
$$
입력 x \\
↓\\
은닉층: z₁ = x @ W₁ + b₁\\
↓ (ReLU)\\
h = ReLU(z₁)\\
↓\\
출력층: z₂ = h @ W₂ + b₂\\
↓ (Sigmoid)\\
ŷ = sigmoid(z₂)\\
↓\\
Loss = BCE(ŷ, y)
$$


**1. GRADIENT 계산**

### 출력층 오차 (BCE + Sigmoid)

손실 함수 (Binary Cross Entropy):

$$
L = - \left[ y \log(\hat{y}) + (1 - y) \log(1 - \hat{y}) \right]
$$

출력층에서의 예측값은 sigmoid 함수:

$$
\hat{y} = \sigma(z_2) = \frac{1}{1 + e^{-z_2}}
$$

Chain Rule 적용:

$$
\frac{\partial L}{\partial z_2} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial z_2}
$$

각 항 미분:

1. BCE 미분:

$$
\frac{\partial L}{\partial \hat{y}} = -\frac{y}{\hat{y}} + \frac{1 - y}{1 - \hat{y}}
$$

2. Sigmoid 미분 (1+e^{-z_2}를 t로 치환 후 미분):

$$
\frac{\partial \hat{y}}{\partial z_2} = \hat{y}(1 - \hat{y})
$$

두 미분을 곱하면:

$$
\frac{\partial L}{\partial z_2} =
\left( -\frac{y}{\hat{y}} + \frac{1 - y}{1 - \hat{y}} \right)
\cdot \hat{y}(1 - \hat{y})
$$

이걸 정리하면:

$$
\frac{\partial L}{\partial z_2} = \hat{y} - y
$$

출력층의 오차는 다음과 같이 계산할 수 있음 (BCE + Sigmoid 많이 나오는 조합)

$$
\delta_2 = \frac{\partial L}{\partial z_2} = \hat{y} - y
$$


### ✅ 가중치 기울기: $\frac{\partial L}{\partial W_2}$

체인 룰을 이용해:

$$
\frac{\partial L}{\partial W_2}
= \frac{\partial L}{\partial z_2} \cdot \frac{\partial z_2}{\partial W_2}
= h^T \cdot \delta_2
$$

### ✅ 편향 기울기: $\frac{\partial L}{\partial b_2}$

편향은 각 샘플마다 동일하게 적용되므로, 오차를 단순히 합산:

$$
\frac{\partial L}{\partial b_2} = \sum_{i=1}^{N} \delta_2^{(i)}
$$

### 은닉층 오차 (Relu)

은닉층의 선형 계산:

$$
z_1 = x \cdot W_1 + b_1
$$

활성화 함수로 ReLU 사용:

$$
h = \text{ReLU}(z_1)
$$


출력층에서 계산한 오차 $\delta_2$를 기반으로  
은닉층으로 오차를 전파 (chain rule):

$$
\delta_1 = \left( \delta_2 \cdot W_2^T \right) \circ \text{ReLU}'(z_1)
$$

- $\text{ReLU}'(z_1)$: $z_1 > 0$일 때는 1, 아니면 0


### ✅ 가중치 기울기: $\frac{\partial L}{\partial W_1}$

은닉층 가중치의 기울기는:

$$
\frac{\partial L}{\partial W_1}
= \frac{\partial L}{\partial z_2}
\cdot \frac{\partial z_2}{\partial h}
\cdot \frac{\partial h}{\partial z_1}
\cdot \frac{\partial z_1}{\partial W_1}
$$

정리해서,

$$
\delta_1 = \left( \delta_2 \cdot W_2^T \right) \circ \text{ReLU}'(z_1)
$$

$$
\frac{\partial L}{\partial W_1} = x^T \cdot \delta_1
$$



### ✅ 편향 기울기: $\frac{\partial L}{\partial b_1}$

편향은 샘플별로 동일하게 적용되므로 오차를 합산:

$$
\frac{\partial L}{\partial b_1} = \sum_{i=1}^{N} \delta_1^{(i)}
$$

**2. PARAMETER 업데이트**

학습률 $\eta$를 이용한 파라미터 업데이트:

$$
W_2 \leftarrow W_2 - \eta \cdot \frac{\partial L}{\partial W_2}
$$

$$
b_2 \leftarrow b_2 - \eta \cdot \frac{\partial L}{\partial b_2}
$$

$$
W_1 \leftarrow W_1 - \eta \cdot \frac{\partial L}{\partial W_1}
$$

$$
b_1 \leftarrow b_1 - \eta \cdot \frac{\partial L}{\partial b_1}
$$



---

**출력층 오차: dL/dz2 = (y_pred - y) / 4**




```
[[ 0.0886],
 [-0.1172],
 [-0.1234],
 [ 0.1644]]
```

**출력층 기울기**

```
dW2 = [[0.0032, 0.0617]]
db2 = 0.0123
```

**은닉층 기울기**

```
dW1 = [[ 0.0200,  0.0190],
       [ 0.0068,  0.0078]]
db1 = [ 0.0246, -0.0126]
```

---

## 🛠️ 3. 파라미터 업데이트 (lr = 0.1)

```
W1 = [[-0.401, -0.459],
      [ 0.605,  0.698]]

b1 = [0.653, -0.050]

W2 = [[-0.647, 0.655]]
b2 = [-0.177]
```

---

## ✅ Epoch 1 결과 요약

| 파라미터 | Epoch 0                        | Epoch 1                        |
|----------|----------------------------------|----------------------------------|
| W1       | [[-0.399, -0.457], [0.606, 0.699]] | [[-0.401, -0.459], [0.605, 0.698]] |
| b1       | [0.655, -0.051]                   | [0.653, -0.050]                   |
| W2       | [[-0.647, 0.661]]                 | [[-0.647, 0.655]]                 |
| b2       | [-0.176]                          | [-0.177]                          |
